In [1]:
import pandas as pd
import numpy as np
cancer = pd.read_parquet('./Binary/data/cancer.parquet')
cancer['mean_rad_gt_mean'] = cancer['mean_radius'] > cancer['mean_radius'].mean()
cancer.head()

,has_cancer,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,mean_rad_gt_mean
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,True
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,True
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,True
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,False
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,True


In [2]:
from sbDtypes import ColDType

In [3]:
cancer.has_cancer

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: has_cancer, Length: 569, dtype: int64

In [4]:
df = pd.read_parquet('./binary_tests.parquet')
df['missing_ints'] = df['ints'].apply(lambda x: None if ((x % 2 == 0) and (np.random.random() <= 0.5)) else x)
df['missing_chars1'] = df['chars1'].apply(lambda x: None if x == 'yes' else x)
df.head()

,ints,chars1,chars3,bools,missing_ints,missing_chars1
0,1,yes,true,True,1.0,None
1,1,yes,true,True,1.0,None
2,1,yes,true,True,1.0,None
3,1,yes,true,True,1.0,None
4,1,yes,true,True,1.0,None


In [5]:
df.to_parquet('./binary_tests.parquet', index=False)

In [6]:
df.missing_ints.value_counts(dropna=False)

missing_ints
1.0    212
0.0    180
NaN    177
Name: count, dtype: int64

In [7]:
for c in df.columns.tolist():
    # if c not in ['chars2', 'chars4']:
    tempcol = ColDType(df[c])
    print(f"{c}: {tempcol.sb_dtype(return_=True)}")
    assert tempcol.sb_dtype(return_=True) == 'binary', \
        f"Column {c} should be binary but is {tempcol.sb_dtype(return_=True)}"

AttributeError: 'NoneType' object has no attribute 'isin'

In [ ]:
col = ColDType(df.missing_ints)
col

ColDType(missing_ints)

In [ ]:
col.s.value_counts(dropna=False)

missing_ints
1.0    212
NaN    183
0.0    174
Name: count, dtype: int64

In [ ]:
col.s.drop_duplicates().reset_index(drop=True)

0    1.0
1    0.0
2    NaN
Name: missing_ints, dtype: float64

In [ ]:
col.get_unique_values() 

0    1.0
1    0.0
2    NaN
Name: missing_ints, dtype: float64

In [ ]:
col.s.name

'missing_ints'

In [ ]:
col.is_type_known

False

In [ ]:
col.is_binary

In [ ]:
col.get_unique_values()

0    1.0
1    0.0
2    NaN
Name: missing_ints, dtype: float64

In [ ]:
col.sb_dtype(return_=True)

'binary'

In [ ]:
col.is_binary

True

In [ ]:
col.s_fmt

In [ ]:
col.format_binary()

In [ ]:
col.s.value_counts(dropna=False)

missing_ints
1.0    212
0.0    174
Name: count, dtype: int64

In [ ]:
col.s_fmt.value_counts(dropna=False)

missing_ints
1    212
0    174
Name: count, dtype: int64